# BME 590 - Workshop 2 - Liquid Handling
**Professor:** Emma Chory, Ph.D.

**Authors:** 
Rick Wierenga, Joe Laforet, Stefan Golas, Ben Perry

---

## Liquid Handling Basics
Now that you have gotten an introduction to setting up a deck in PyLabRobot, this second lesson is going to cover everything liquids! From setting their inintial levels to moving them around to managing tips and cross-contamination!


### Adding Liquids to Lab Objects
Usually when you start an automated protocol, you will have reservoirs of liquid somewhere on the deck. In fancier setups, this liquid can be moved to the deck from a pump system or a robotic arm carrying a reservoir from temperature control. For these exercises we will assume the liquid was placed manually by the user (such as manual preparation of reagents or usage of a kit, as is common in many experiments)

Therefore, to begin, we will need to **set the state** of liquids in a starting set of wells or a reservoir.

To start, let's go ahead and set up a basic HamiltonSTAR deck with:

- 1 reservoir
- 1 12 tube rack
- 2 96 well plates
- 1 384 well plate
- 1 1000 uL tip rack
- 1 300 uL tip rack
- 1 10 uL tip rack

We will start with a function that will set up this deck for us. Run the code below and you should get a deck that looks like this!

<div>
<img src="../figs/liquid_deck.png" width="850"/>
</div>

In [100]:
from pylabrobot.resources import Deck
from pylabrobot.liquid_handling.backends.backend import LiquidHandlerBackend
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import LiquidHandlerChatterboxBackend
from pylabrobot.resources.hamilton import STARDeck
from pylabrobot.visualizer.visualizer import Visualizer

from pylabrobot.resources import (
    set_tip_tracking,
    set_volume_tracking,
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    corning_96_wellplate_360ul_flat,
    corning_384_wellplate_112ul_flat,
    nest_1_reservoir_195ml,
    corning_12_wellplate_6point9ml_flat,
    HTF,
    LTF,
    STF
)
set_volume_tracking(enabled=True)
set_tip_tracking(enabled=True)

async def visualize_deck(deck: Deck,
                         backend: LiquidHandlerBackend):
    try:
        lh = LiquidHandler(backend=backend, deck=deck)
        vis = Visualizer(resource = lh)
        await lh.setup()
        await vis.setup()
        return lh
    except Exception as e:
        print(f"Error! Got excpetion: {e}")

async def make_liquid_handling_deck():
    deck = STARDeck()
    tip_carrier = TIP_CAR_480_A00(name="awesome tip carrier 96x5")
    plate_carrier = PLT_CAR_L5AC_A00(name = "awesome plate carrier")
    tip_rack_names = [
        "1000_tips_0",
        "1000_tips_1",
        "300_tips_0",
        "300_tips_1",
        "10_tips_0"
    ]
    tip_rack_instantiators = [
        HTF,
        HTF,
        STF,
        STF,
        LTF
    ]
    plate_names = [
        "reservoir_0",
        "12_plate_0",
        "96_plate_0",
        "96_plate_1",
        "384_plate_0"
    ]
    plate_instantiators = [
        nest_1_reservoir_195ml,
        corning_12_wellplate_6point9ml_flat,
        corning_96_wellplate_360ul_flat,
        corning_96_wellplate_360ul_flat,
        corning_384_wellplate_112ul_flat
    ]
    for i, (name, tip_rack) in enumerate(zip(tip_rack_names, tip_rack_instantiators)):
        tip_carrier[i] = tip_rack(name = name)
    for i, (name, plate) in enumerate(zip(plate_names, plate_instantiators)):
        plate_carrier[i] = plate(name = name)

    deck.assign_child_resource(plate_carrier, rails = 5)
    deck.assign_child_resource(tip_carrier, rails = 11)
    return deck

deck = await make_liquid_handling_deck()
lh = await visualize_deck(deck, LiquidHandlerChatterboxBackend())

Setting up the liquid handler.
Resource deck was assigned to the liquid handler.
Resource trash was assigned to the liquid handler.
Resource trash_core96 was assigned to the liquid handler.
Resource waste_block was assigned to the liquid handler.
Resource awesome plate carrier was assigned to the liquid handler.
Resource awesome tip carrier 96x5 was assigned to the liquid handler.
Websocket server started at http://127.0.0.1:2140
File server started at http://127.0.0.1:1356 . Open this URL in your browser.


Let's print a summary of our deck so far.

In [2]:
print(lh.deck.summary())

Rail  Resource                         Type           Coordinates (mm)
(-6)  ├── trash_core96                 Trash          (-58.200, 106.000, 229.000)
      │
(5)   ├── awesome plate carrier        PlateCarrier   (190.000, 063.000, 100.000)
      │   ├── reservoir_0              Plate          (194.000, 071.500, 181.600)
      │   ├── 12_plate_0               Plate          (194.000, 167.500, 183.660)
      │   ├── 96_plate_0               Plate          (194.000, 263.500, 182.600)
      │   ├── 96_plate_1               Plate          (194.000, 359.500, 182.600)
      │   ├── 384_plate_0              Plate          (194.000, 455.500, 183.360)
      │
(11)  ├── awesome tip carrier 96x5     TipCarrier     (325.000, 063.000, 100.000)
      │   ├── 1000_tips_0              TipRack        (331.200, 073.000, 214.950)
      │   ├── 1000_tips_1300_tips_0    TipRack        (331.200, 169.000, 214.950)
      │   ├── 300_tips_1               TipRack        (331.200, 265.000, 214.950)
      │   ├

#### Setting Liquid Volumes
In order to manually set liquid volumes, there are a couple of methods we can use. Since we are going to be adding liquids to the plates/reservoirs, let's go ahead and get the 12-well and 96-well plate using `get_resource` with their names.

In [3]:
plate_96 = lh.deck.get_resource("96_plate_0")
plate_12 = lh.deck.get_resource("12_plate_0")

The easiest way we can fill a specific level of liquid in a specific well is to access it by index.

Remember that a plate has wells as it's children, and we can access these by identifier using the `get_well` method. This method expects an identifier, which can be either an index (0 - 95) representing the well index, or it can be the actual letter-integer coordinate of the well (e.g. A1, B2, etc.)

We show both indexing methods below

In [10]:
well_1 = plate_96.get_well(0)
well_2 = plate_96.get_well("A1")
well_3 = plate_96.get_item("A1")
well_4 = plate_96.get_item(0)

print(well_1 == well_2 == well_3 == well_4) # this should evaluate to True, meaning these are all equivalent methods.

True


To fill this well with liquid, we need to access its `tracker` attribute and specifically use the `set_liquids` method. This method expects a **tuple**, where the first index is the name or class of liquid and the second index is the volume, in uL, for example:
- **(None, 180)** - Unknown liquid - 180 uL volume
- **("Red Dye", 2000)** - Custom liquid named "Red Dye" - 2000 uL volume
- **(Liquid.OCTANOL, 360)** - Official PLR Liquid class for Octanol - 360 uL volume.

Let's go ahead and demonstrate some of these below

In [7]:
from pylabrobot.resources import Liquid # import for liquid class

# get wells of interest
unknown_liquid = (None, 180)
red_dye = ("Red Dye", 2000)
octanol = (Liquid.OCTANOL, 360)

plate_96.get_well("A1").tracker.set_liquids([unknown_liquid])
plate_96.get_well("B1").tracker.set_liquids([octanol])
plate_12.get_well("A1").tracker.set_liquids([red_dye])

Great! Now go look at the visualizer. You should see something like this:

<div>
<img src="../figs/filled_wells.png" width="250"/>
</div>

The visualizer will actually shade the liquid volume by the amount in a well, relative to that well's maximum volume! So you can see, well B1 is filled to capacity (360 uL / 360 uL), so it has the deepest red color, while well A1 only has 180 uL, so it is filled to half capacity. Finally, well A1 of the 12-well plate is filled the least (percentage-wise) so it has the lightest color.

Currently, only red is supported as the color of solvent, so you'll haave to use your imagination here if mixing liquids. We can also use the `add_liquid` and `remove_liquid` functions to achieve similar functionality as the `set_liquids` function. These should **not** be used in lieu of pipetting operations, but rather are meant to be used to represent manual operations outside the liquid handler.

**Note-** The `add_liquid` function expects the liquid identifier as the first argument and the amount as the second argument, so just unpack the tuple. The `remove_liquid` doesn't require a liquid identifier, just the amount ot remove. The `remove_liquid` function will lso return a List of Tuples of liquids and volume removed if you call the function, which may be useful to store as a variable, for example, if you have to manually move an entire well contents somewhere else manually during a protocol.

In [8]:
plate_96.get_well("A1").tracker.add_liquid(*unknown_liquid)
plate_12.get_well("A1").tracker.remove_liquid(volume = 2000)

[('Red Dye', 2000)]

Great! You should now have something that looks like this

<div>
<img src="../figs/add_and_remove_liquid.png" width="250"/>
</div>

Note, since we doubled the liquid in well A1, it is now also full! We have also removed all the liquids in well A1 of the 12-well plate.

Certainly, you could manually assign liquids to each well, but this would get cumbersome with a lot of wells you are trying to fill!

There are several options for traversing over wells using iterators. Let's see some examples below

#### Traversal of Labware
There are several ways to iterate over a given piece of labware, including **manual loops**, **labware-specific methods**, and **generators**. This section will review all three methods; you should implement the one you are most comfortable with using in the Python code (i.e. there is not a definite solution on how to traverse over labware.)

##### Manual Looping
Since we know how many wells are preseent on a given piece of labware, we can utilize that information to manually loop through all spots on a given plate using a **for loop** in Python and the `get_well()` method.


In [22]:
# define function to reset liquids on a plate
def reset_liquids(plate, plate_num_items: int = 96):
    liquids = [[(None, None)]] * plate_num_items
    plate.set_well_liquids(liquids)

In [65]:
# reset deck
deck = await make_liquid_handling_deck()
lh = await visualize_deck(deck, LiquidHandlerChatterboxBackend())

# get the plates of interest
plate_96_0 = lh.deck.get_resource("96_plate_0")
plate_96_1 = lh.deck.get_resource("96_plate_1")
plate_12 = lh.deck.get_resource("12_plate_0")

# lets add 150 uL to every well on the 96 well plate
for i in range(96):
    plate_96_0.get_well(i).tracker.set_liquids([red_dye])

Setting up the liquid handler.
Resource deck was assigned to the liquid handler.
Resource trash was assigned to the liquid handler.
Resource trash_core96 was assigned to the liquid handler.
Resource waste_block was assigned to the liquid handler.
Resource awesome plate carrier was assigned to the liquid handler.
Resource awesome tip carrier 96x5 was assigned to the liquid handler.
Websocket server started at http://127.0.0.1:2134
File server started at http://127.0.0.1:1350 . Open this URL in your browser.


In [24]:
# lets add 150 uL but only to every other column on the second 96 well plate
for i in range(96):
    if i % 16 <= 7: # modulo operator get's the remainder of each item
        plate_96_1.get_well(i).tracker.set_liquids([red_dye])

In [25]:
# we can also index by the letter-integer combination. let's fill in the second row of the second plate
# in python, an f-string is denoted by f"<str_contents>" where you can fill in variables inside the quotes
# with their value. for example print(f"Number: {i}") will print "Number 1, Number 2..."

# 2nd row is B1 - B12
# 4th row is D1 - D12
reset_liquids(plate_96_1)
for i in range(12):
    plate_96_1.get_well(f"B{i + 1}").tracker.set_liquids([red_dye])
    plate_96_1.get_well(f"D{i + 1}").tracker.set_liquids([red_dye])

In [ ]:
# now let's fill in the second and 4th columns
# we can do this by defining a capital letter alphabet that ranges from A - H
# 2nd column is A2 - H2
# 4th column is A4 - H4
reset_liquids(plate_96_1)
alphabet = [chr(i) for i in range(65, 73)]
print(f"{alphabet=}") # prints alphabet=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
for i in range(8):
    plate_96_1.get_well(f"{alphabet[i]}2").tracker.set_liquids([red_dye])
    plate_96_1.get_well(f"{alphabet[i]}4").tracker.set_liquids([red_dye])

alphabet=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']


In [43]:
# if you don't know the number of items in a given plate, you can check
print(f"X-axis items (96 well plate): {plate_96_1.num_items_x}")
print(f"Y-axis items (96 well plate): {plate_96_1.num_items_y}")
print(f"All items (96 well plate): {plate_96_1.num_items}")

print(f"X-axis items (12 well plate): {plate_12.num_items_x}")
print(f"Y-axis items (12 well plate): {plate_12.num_items_y}")
print(f"All items (12 well plate): {plate_12.num_items}")

# you can also access plates by row or column as well
# generally, you shouldn't use the num_items methods because you should know how many
# rows and columns are in your experimental setup a priori.
reset_liquids(plate_96_1)
num_rows = plate_96_1.num_items_y
num_cols = plate_96_1.num_items_x
for row in range(num_rows):
    if row % 2 == 0:
        for well in plate_96_1.row(row):
            well.tracker.set_liquids([red_dye])

X-axis items (96 well plate): 12
Y-axis items (96 well plate): 8
All items (96 well plate): 96
X-axis items (12 well plate): 4
Y-axis items (12 well plate): 3
All items (12 well plate): 12


#### Traversal Functions
To iterate over locations on labware, we can also use the `traverse()` function. This produces a [generator](https://wiki.python.org/moin/Generators) object that we can iterate over to yield our desired wells.

`traverse()` takes in three arguments:
* **batch_size** is the amount of wells to return
* **start** is where to start
* **direction** is how to iterate over the wells.

**More info on direction:**
- starting location can be one of "top_left", "bottom_left", "top_right", "bottom_right"
- direction can be one of "up", "down", "right", "left", "snake_up", "snake_down","snake_left", "snake_right",

The snake directions alternate between going in the given direction and going in the opposite direction. For example, "snake_down" will go from **A1** to **H1**, then **H2** to **A2**, then **A3** to **H3**, etc.

Enables complex traversal methods without too much troublesome python code.

In [146]:
plate = plate_96_0
wells = plate.traverse(start="bottom_right", batch_size = 4, direction='down')

# this function will help print the wells being iterated over in each batch.
# it is recommended to try different combinations of batch_size, direction, and starting point to visualize
# how this affects the well traversal.
def print_well_batches(well_batch_iterator):
    for i, well_batch in enumerate(well_batch_iterator):
        start_well = well_batch[0].name.split("_")[-1]
        end_well = well_batch[-1].name.split("_")[-1]
        print(f"Batch {i}:\tStart Well: {start_well}\tEnd Well: {end_well}")

print_well_batches(wells)

ValueError: Cannot start from bottom_right and go down.

In [145]:
plate = plate_96_0
wells = plate.traverse(start="top_left", batch_size = 4, direction='down')

# this function will enable trying differnet traversal methods and filling every other batch with
# red liquid on the visualizer to see how the traversal method changes things
def fill_every_other_batch(well_batch_iterator, plate):
    reset_liquids(plate)
    for i, well_batch in enumerate(well_batch_iterator):
        if i % 2 == 0:
            for well in well_batch:
                well.tracker.set_liquids([red_dye])
fill_every_other_batch(wells, plate)

#### Tip racks
Tip racks are similar to a plate with wells or any other iterable resource; however, instead of caring about **what liquids** and **how much** are in each well, we now simply ask the question whether or not a **tip spot** contains an **unused tip**.

Generally, most liquid handlind movements will necessarily have the following steps at some point:

1. Pick up $N$ unused tips (or alternatively, re-use tips if they will not lead to cross-contamination...more on this later).

2. Aspirate $V$ volume of liquid into the $N$ channels holding $N$ tips.

3. Move pipetting head to a separate plate or target destination.

4. Dispense $V$ volume of liquid into $N$ target wells with $N$ channels.

5. Discard the used tips in the trash container on your machine.

Let's examine our tip racks and some utilities when using them.

In [101]:
deck = await make_liquid_handling_deck()
lh = await visualize_deck(deck, LiquidHandlerChatterboxBackend())
print(deck.summary)

# get the plates of interest
tips_1000_uL_0 = lh.deck.get_resource("1000_tips_0")
tips_1000_uL_1 = lh.deck.get_resource("1000_tips_1")
tips_300_uL = lh.deck.get_resource("300_tips_1")

Setting up the liquid handler.
Resource deck was assigned to the liquid handler.
Resource trash was assigned to the liquid handler.
Resource trash_core96 was assigned to the liquid handler.
Resource waste_block was assigned to the liquid handler.
Resource awesome plate carrier was assigned to the liquid handler.
Resource awesome tip carrier 96x5 was assigned to the liquid handler.
Websocket server started at http://127.0.0.1:2141
File server started at http://127.0.0.1:1357 . Open this URL in your browser.
<bound method HamiltonDeck.summary of HamiltonSTARDeck(name=deck, location=Coordinate(000.000, 000.000, 000.000), size_x=1900, size_y=653.5, size_z=900, category=deck)>


To “empty” a tip rack after initialization, use the `empty()` method. To “fill” a tip rack after initialization, use the `fill()` method.

In [81]:
tips_1000_uL_0.empty()
tips_300_uL.empty()

In [82]:
tips_1000_uL_0.fill()
tips_300_uL.fill()

You can have the liquid handler pick up a tip by calling `lh.pick_up_tips()`, and passing in the `TipSpot`s of the tips you want to retrieve. `TipSpot`s are indexed the same way that wells are.

Let's try it

In [102]:
await lh.pick_up_tips(tip_spots = tips_1000_uL_0["A1"],
                use_channels = [0])

Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: 1000_tips_0_tipspot_0_0 0,0,0            HamiltonTip  1065             8                    95.1             Yes       


Returning tips is useful when:

1.  The tips are clean and are being put back where they were retrieved from

2.  The tips are not clean but can be **re-used** with the same liquid.

Simply call `lh.return_tips()` to return the tips back to their starting point.

In [104]:
# return tips
await lh.return_tips()

RuntimeError: No tips have been picked up.

What happens if we call `lh.return_tips()` again? We should run into a **RuntimeError** telling us that no tips have been picked up, so it makes no sense to try to return tips if there are no tips to return.

PyLabRobot has several very useful errors like this that will help you plan a protocol (more on this later)

Try it out below. Call `lh.return_tips()`

In [106]:
# error: return tips - explanation
await lh.return_tips()

RuntimeError: No tips have been picked up.

PyLabRobot can also track whether or not the liquid handler has a tip, based on the history of executed steps on the liquid handler.

In [ ]:
# lh tip state
print(lh.head[0].has_tip)

False


We used the `[0]` syntax because we are asking if a the first channel has a tip on it on the given liquid handler. To see the full status of the head of the liquid handler, we can print `lh.head`

In [133]:
print(lh.head)

{0: TipTracker(Channel 0, is_disabled=False, has_tip=True tip=HamiltonTip(HIGH_VOLUME, has_filter=True, maximal_volume=1065, fitting_depth=8, total_tip_length=95.1, pickup_method=OUT_OF_RACK), pending_tip=HamiltonTip(HIGH_VOLUME, has_filter=True, maximal_volume=1065, fitting_depth=8, total_tip_length=95.1, pickup_method=OUT_OF_RACK)), 1: TipTracker(Channel 1, is_disabled=False, has_tip=False tip=None, pending_tip=None), 2: TipTracker(Channel 2, is_disabled=False, has_tip=False tip=None, pending_tip=None), 3: TipTracker(Channel 3, is_disabled=False, has_tip=False tip=None, pending_tip=None), 4: TipTracker(Channel 4, is_disabled=False, has_tip=False tip=None, pending_tip=None), 5: TipTracker(Channel 5, is_disabled=False, has_tip=False tip=None, pending_tip=None), 6: TipTracker(Channel 6, is_disabled=False, has_tip=False tip=None, pending_tip=None), 7: TipTracker(Channel 7, is_disabled=False, has_tip=False tip=None, pending_tip=None)}


This is hard to read, let's write a function to print out this information is a more user-friendly way.

In [138]:
def print_channels_tip_origin(lh):

    # Prints the origin location of all tips currently on the robot
    cur_pipetter = lh.head
    print("\nChannel Status...\n\n")

    for channel in cur_pipetter:
        print(f"Channel {channel}:")
        tip_tracker = lh.head[channel]
        if tip_tracker.has_tip == True:
            print(f"Tip: {tip_tracker.get_tip()}")
            print(f"Origin: {tip_tracker.get_tip_origin()}")
        else:
            print("No tip present.")
        print()

print_channels_tip_origin(lh)


Channel Status...


Channel 0:
Tip: HamiltonTip(HIGH_VOLUME, has_filter=True, maximal_volume=1065, fitting_depth=8, total_tip_length=95.1, pickup_method=OUT_OF_RACK)
Origin: TipSpot(name=1000_tips_0_tipspot_0_0, location=Coordinate(007.200, 068.300, -83.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot)

Channel 1:
No tip present.

Channel 2:
No tip present.

Channel 3:
No tip present.

Channel 4:
No tip present.

Channel 5:
No tip present.

Channel 6:
No tip present.

Channel 7:
No tip present.



In [129]:
async def reset_tip_box_and_pipette_head(lh, tip_rack):
    print("Resetting tip rack...")
    tip_rack.fill()
    await lh.discard_tips()

Great, now we can see, of the **8 channels** on our pipeeter, that none of them contain a tip. Let's pick up a tip and then try again.

In [139]:
await reset_tip_box_and_pipette_head(lh, tips_1000_uL_0)
await lh.pick_up_tips(tip_spots = tips_1000_uL_0["A1"],
                use_channels = [0])
print_channels_tip_origin(lh)


Resetting tip rack...
Dropping tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: trash                0,0.0,0          HamiltonTip  1065             8                    95.1             Yes       
Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: 1000_tips_0_tipspot_0_0 0,0,0            HamiltonTip  1065             8                    95.1             Yes       

Channel Status...


Channel 0:
Tip: HamiltonTip(HIGH_VOLUME, has_filter=True, maximal_volume=1065, fitting_depth=8, total_tip_length=95.1, pickup_method=OUT_OF_RACK)
Origin: TipSpot(name=1000_tips_0_tipspot_0_0, location=Coordinate(007.200, 068.300, -83.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot)

Channel 1:
No tip present.

Channel 2:
No tip present.

Channel 3:
No tip present.

Channel 4:
No tip present.

Channel 5:
No tip presen

Great! Now what you should notice is that channel 0 now contains both:

- A tip: `HamiltonTip(HIGH_VOLUME, has_filter=True, maximal_volume=1065, fitting_depth=8, total_tip_length=95.1, pickup_method=OUT_OF_RACK)`

- The tip's origin: `TipSpot(name=1000_tips_0_tipspot_0_0, location=Coordinate(007.200, 068.300, -83.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot)`

So far, we have only shown how to get one tip at a time. For robots with only **one channel**, this is all you are limited to, and pipetting operations must be singleton (i.e. meaning, you would have to iterate over wells for pipetting with a **for loop**)

For coding purposes, this won't make much of a difference, since **for loops** can execute as fast as your machine can process the Pythonic code. However, if your particular liquid handler can grab **multiple tips** at the same time, then the number of **physical operations** (movement of the robotic arm) decreases signficantly, which can not only speed up your protocol, but also had other benefits such as energy cost savings and reduced wear-and-tear on the physical system.

The `use_channels` argument is very useful here, where you want to pass in a list of **channel indices** that you'd like to use to pick up the tips.

In [141]:
# pick up tips with 2 channels
await reset_tip_box_and_pipette_head(lh, tips_1000_uL_0)
await lh.pick_up_tips(tip_spots = tips_1000_uL_0["A1", "A2"],
                use_channels = [0, 1])
print_channels_tip_origin(lh)

Resetting tip rack...
Dropping tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: trash                0,4.5,0          HamiltonTip  1065             8                    95.1             Yes       
  p1: trash                0,-4.5,0         HamiltonTip  1065             8                    95.1             Yes       
Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: 1000_tips_0_tipspot_0_0 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p1: 1000_tips_0_tipspot_1_0 0,0,0            HamiltonTip  1065             8                    95.1             Yes       

Channel Status...


Channel 0:
Tip: HamiltonTip(HIGH_VOLUME, has_filter=True, maximal_volume=1065, fitting_depth=8, total_tip_length=95.1, pickup_method=OUT_OF_RACK)
Origin: TipSpot(name=1000_tips_0_ti

In [143]:
# using all 8 channels
await reset_tip_box_and_pipette_head(lh, tips_1000_uL_0)
tip_spots = [f"A{i + 1}" for i in range(8)]
channels = [i for i in range(8)]
await lh.pick_up_tips(tip_spots = tips_1000_uL_0[tip_spots],
                use_channels = channels)
print_channels_tip_origin(lh)

Resetting tip rack...


RuntimeError: No tips have been picked up and no channels were specified.

some devices even have a 96-tip head

In [ ]:
# pick up 96

you can even use this for quadrant mode

In [ ]:
# quadrant mode

generators and iterators over tips

In [ ]:
# random selection

In [ ]:
# iterator

### Liquid Handling - Aspiration and Dispensing
After you have picked up tips, you then need to use this step

simple transfer

In [ ]:
# transfer from one well to another

simple transfer with for loop

In [ ]:
# example

simple transfer with channels

In [ ]:
# example

transfer with 96 head

#### Setting Tip Locations
##### Tip Fill/Removal


#### Traversal of Labware Spots
##### Universal Traversal

#### Aspiration Operations

#### Dispense Operations

#### Tip Return & Discarding

#### Trackers
Trackers in PyLabRobot are objects that keep track of the state of the deck throughout a protocol. Three types of trackers currently exist:

- **tip trackers** - tracks the presence of tips in tip racks and on the pipetting channels
- **volume trackers** - tracks the volume in pipetting tips and wells/reservoirs
- **cross contamination** - tracks whether a given action would result in two liquids mixing unintentionally, thus introducing contamination in a sample or tip.

Why do we need trackers? 
#### Tracker Error Demonstration

#### Other Errors 

#### Practice Time - Moving Liquids

#### Exercises